# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Изучение общей информации

Начнем знакомство с данными с импорта библиотеки pandas, затем прочитаем таблицу и сохраним ее в переменной `df`. Выведем первые десять строк таблицы, чтобы иметь общее представление о происходящем. Также выведем общую информацию о таблице командой `df.info()`

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display(df.head(10))
print()
df.info()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, мы имеем таблицу из 12 столбцов и 21525 строк. Типы данных: `int64`, `float64`, `object`.

Согласно документации, столбцы имеют следующее значение:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Все столбцы названы в рамках хорошего стиля, коррекция не требуется.

Теперь ознакомимся с содержанием каждого столбца, чтобы выявить задачи для предобработки данных.


Столбец `'children'` содержит 8 уникальных значений. Среди них явно выделяются значения `-1` и `20`. Отрицательного количества детей быть не может, да и 20 детей на семью это перебор, тем более, для 76 семей. Значит, будем разбираться.

In [2]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В колонке `'days_employed'` сразу бросаются в глаза три момента - отрицательные зачения, тип данных `float` и шестизначные числа. 

Посчитаем, сколько у нас отрицательных значений в столбце:

In [3]:
print(df[df['days_employed']<0]['days_employed'].count())


15906


Получилось почти 16000, это явно какая-то недоработка при составлении таблицы, вероятно, неправильная формула при вводе стажа. 

Теперь шестизначные числа. Немного забегая вперед, даже если предположить, что человек работал с 18 до 75 лет (возраст самого старого заемщика) без перерывов, то его максимальный стаж в днях составит `365*56=20440` дней(плюс-минус, високосные года не берем). 

Посмотрим, сколько у нас таких заемщиков:

In [4]:
print(df[df['days_employed']>20440]['days_employed'].count())

3445


Почти 3500 человек. Явно что-то пошло не так. Пренебречь этими данными мы не можем, так как их слишком много. В идеале надо разбираться с составителями таблицы, откуда взялись эти значения. Вероятно, это стаж в часах, или неправильно внесенные из других источников данные. Но так как для нашего исследования этот столбец не представляет интереса, можно оставить все как есть.

Третья странность - это некорректный тип данных, потому что учитываются только целые дни. В рамках выполнения задания дальше по ходу исследования мы поменяем тип данных в этом столбце.

Теперь обратимся к столбцу `'dob_years'`. Посчитав значения, мы увидим, что 101 человек у нас без возраста. Так не бывает, по какой-то причине данные были заполнены нолями, разберемся с этим на этапе предобработки.

In [5]:
print(df['dob_years'].value_counts().sort_index())

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


Переходим к столбцу `'education'`. Сразу бросается в глаза разный регистр написания. Еще одна задача на предобработку.

In [6]:
print(df['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Значений в столбце `'education_id'` у нас 5, что совпадает с уникальными значениями из предыдущего столбца (среднее, неоконченное высшее, высшее, начальное, ученая степень), тут все в порядке.

In [7]:
print(df['education_id'].value_counts().sort_index())

0     5260
1    15233
2      744
3      282
4        6
Name: education_id, dtype: int64


Посчитаем уникальные значения семейного положения в столбце `'family_status'`. Все логично, все понятно. Мелкая помарка - статус `Не женат / не замужем` начинается с заглавной буквы, уберем на предобработке.

In [8]:
print(df['family_status'].value_counts()) 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Значений у нас 5, что совпадает с предыдущим столбцом, тут все в порядке.

In [9]:
print(df['family_status_id'].value_counts().sort_index())

0    12380
1     4177
2      960
3     1195
4     2813
Name: family_status_id, dtype: int64


В столбце со значениями пола есть только одно пропущенное значение. Одной строкой можно пренебречь.

In [10]:
print(df['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


По столбцу с типом занятости никаких странных моментов тоже не наблюдается. В основном, наемные работники.

In [11]:
print(df['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


В столбце с задолженностями все сходится. Должников не так уж и много.

In [12]:
print(df['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


В столбце с ежемесячным доходом нет смысла считать уникальные значения, доход у всех разный. Проверим только, чтобы не было отрицательных или нулевых значений.

In [13]:
print("Заемщиков с нулевым доходом: {}".format(df[df['total_income']<=0]['total_income'].count()))


Заемщиков с нулевым доходом: 0


По значениям столбца `'purpose'` мы можем узнать, на что же берутся кредиты. На первый взгляд, есть много повторяющихся целей, сформулированных по-разному. Уточнить цели нам поможет лемматизация.

In [14]:
print(df['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

Таблица для исследования содержит в себе большой массив данных, в котором, к сожалению, встречаются пропуски в критичных для исследования столбцах (количество детей, доход). Также для успешного решения поставленных задач необходимо провести работу по выделению целей кредита. 

## Предобработка данных

### Обработка пропусков

Приступим к обработке пропущенных значений проверим количество пропусков в таблице через `isna()`.

In [15]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы видим одинаковое количество пропусков в столбцах `'days_employed'` и `'total_income'`. Вряд ли такое совпадение случайно. попробуем вывести на экран строки, в которых пропущено значение в столбце `'days_employed'`.

In [16]:
display(df[df['days_employed'].isna()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


И правда, мы видим, что значения в столбце `'total_income'` тоже пропущены. 

Прежде, чем заполнять пропуски, приведем все значения столбца `'days_employed'` к положительным значениям, применив `.abs` ко столбцу: 

In [17]:
df['days_employed'] = df['days_employed'].abs()
display(df.head(10))
print()
print("Максимальное количество дней стажа: {}".format(df['days_employed'].max()))
print()
print("Минимальное количество дней стажа: {}".format(df['days_employed'].min()))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи



Максимальное количество дней стажа: 401755.40047533

Минимальное количество дней стажа: 24.14163324048118


И опять-таки мы видим, что максимальное значения явно нереальное: 401755 дней это 1100 лет. Данные требуют уточнения.

В принципе, этот столбец можно удалить, так как он явно не нужен в нашем исследовании, и данные в нем некорректны, но мы его оставим - он может помочь коллегам, которые будут работать с этой таблицей, так как корректные данные в нем тоже наверняка есть. Заполним пропуски в столбце медианным значением:

In [18]:
total_days_avg = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(total_days_avg)
print("Количество пропусков в столбце:'days_employed':{}".format(df['days_employed'].isna().sum()))

Количество пропусков в столбце:'days_employed':0


У нас есть 101 нулевое значение в столбце `'dob_years'`. Удалим эти строки и проверим, с какого возраста теперь начинается возраст заемщиков:

In [19]:
df = df.drop(df[df['dob_years']==0].index)
print(df['dob_years'].value_counts().sort_index().head())

19     14
20     51
21    111
22    183
23    254
Name: dob_years, dtype: int64


Теперь возраст начинается с 19 лет, все в порядке.

Теперь займемся пропусками в столбце `'total_income'`. Взять среднее значение по столбцу будет не совсем корректно, потому что доходы у разных катеогрий замещиков могут различаться, поэтому сначала посчитаем средний доход для каждой группы и составим небольшой датафрейм:

In [20]:
mean_income_group = df.groupby('income_type')['total_income'].mean()
mean_income = pd.DataFrame({'mean_income':mean_income_group}).reset_index()
print(mean_income)

       income_type    mean_income
0      безработный  131339.751676
1        в декрете   53829.130729
2      госслужащий  171008.225815
3        компаньон  202483.482019
4        пенсионер  137120.690001
5  предприниматель  499163.144947
6        сотрудник  161404.247418
7          студент   98201.625314


И правда, мы видим, что средний доход в зависимости от категории заемщика, достаточно сильно различается. 

Теперь присоединим этот датафрейм к таблице `df` отдельным столбцом:

In [21]:
df = df.merge(mean_income, on='income_type', how='left')  # слияние таблиц
display(df.head(5))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_income
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,161404.247418
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,161404.247418
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,161404.247418
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,161404.247418
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,137120.690001


 Заполним пропуски в столбце `'total_income'` значениями из `'mean_income'` и проведем быструю проверку на наличие пропусков:

In [22]:
df['total_income'] = df['total_income'].fillna(df['mean_income'])
print("Количество пропусков в столбце:'total_income':{}".format(df['total_income'].isna().sum()))


Количество пропусков в столбце:'total_income':0


Удалим единственную строку с отсутствующим значением в столбце `'gender'`:

In [23]:
df = df.drop(df[df['gender']=='XNA'].index)
print(df['gender'].value_counts())

F    14164
M     7259
Name: gender, dtype: int64


Значение `20` в столбце с количеством детей теоретически можно объяснить опечаткой при наборе: 2 и 0 на цифровой клавиатре расположены рядом, но мы не знаем, что там было на самом деле, поэтому заменим на среднее значение. А вот с `-1` непонятно, потому что опечататься так достаточно сложно, а что имели в виду авторы таблицы, мы не знаем. Поэтому удаляем строки - там всего 47 позиций, допустимая погрешность.

In [24]:
df = df.drop(df[df['children']== -1].index)
children_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
pd.DataFrame({'Количество детей':df['children'].value_counts()})

,Количество детей
0.0,14154
1.0,4802
2.0,2042
3.0,328
4.0,41
5.0,9


Выводим таблицу после заполнения пропусков, проверяем. По итогу потеряли 347 позиций, при общем количестве более 21000 допустимо.

In [25]:
display(df.head(15))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_income
0,1.0,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,161404.247418
1,1.0,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,161404.247418
2,0.0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,161404.247418
3,3.0,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,161404.247418
4,0.0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,137120.690001
5,0.0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,202483.482019
6,0.0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,202483.482019
7,0.0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,161404.247418
8,2.0,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,161404.247418
9,0.0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,161404.247418


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21376 entries, 0 to 21423
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21376 non-null  float64
 1   days_employed     21376 non-null  float64
 2   dob_years         21376 non-null  int64  
 3   education         21376 non-null  object 
 4   education_id      21376 non-null  int64  
 5   family_status     21376 non-null  object 
 6   family_status_id  21376 non-null  int64  
 7   gender            21376 non-null  object 
 8   income_type       21376 non-null  object 
 9   debt              21376 non-null  int64  
 10  total_income      21376 non-null  float64
 11  purpose           21376 non-null  object 
 12  mean_income       21376 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.3+ MB


**Вывод**

Итого пропусков оказалось не так много, все они нами заполнены, таблица готова к дальнейшему анализу.

### Замена типа данных

Заменим тип данных в столбцах `'days_employed'`, `'total_income'`, `'mean_income'` и выведем информацию. 
Столбец со стажем выбран потому, что нас интересует количество целых дней, столбцы с доходом для упрощения восприятия информации, и потому что копейки в суммах не несут в себе важной информации на общем фоне. Используем метод `astype`, потому что он проще.

In [26]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df['mean_income'] = df['mean_income'].astype('int')
print()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21376 entries, 0 to 21423
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21376 non-null  float64
 1   days_employed     21376 non-null  int64  
 2   dob_years         21376 non-null  int64  
 3   education         21376 non-null  object 
 4   education_id      21376 non-null  int64  
 5   family_status     21376 non-null  object 
 6   family_status_id  21376 non-null  int64  
 7   gender            21376 non-null  object 
 8   income_type       21376 non-null  object 
 9   debt              21376 non-null  int64  
 10  total_income      21376 non-null  int64  
 11  purpose           21376 non-null  object 
 12  mean_income       21376 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.3+ MB


**Вывод**

Смена типа данных прошла успешно, в нужных нам столбцах стоит `int64`.

### Обработка дубликатов

Для начала выделим и удалим явные дубликаты с последующей проверкой:

In [27]:
df = df.drop_duplicates().reset_index(drop=True)
print("Количество пропусков: {}".format(df.duplicated().sum()))

Количество пропусков: 0


Теперь займемся столбцом `'education'`. Всего в нем присутствует три варианта заполнения - строчные буквы, заглавные буквы, и одна заглавная буква в начале. Вероятно, таблица заполнялась разными людьми, или в какой-то период времени менялись требования к оформлению. Приведем все в единообразный вид и проверим:

In [28]:
df['education'] = df['education'].str.lower()  # приведение к нижнему регистру
display(df['education'].head(10))

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
5     высшее
6     высшее
7    среднее
8     высшее
9    среднее
Name: education, dtype: object

In [29]:
df = df.drop_duplicates().reset_index(drop=True)  # еще раз прогоняем проверку, чтобы не появилось новых дубликатов
print("Количество пропусков: {}".format(df.duplicated().sum()))

Количество пропусков: 0


Похожая проблема у нас была в столбце `'family_status'` со статусом `Не женат / не замужем`. Тут это больше похожа на опечатку, потому что встречается только в одном месте. Тем не менее, аналогично поправляем и этот столбец, затем проверяем. 

In [30]:
df['family_status'] = df['family_status'].str.lower()  # приведение к нижнему регистру
display(df['family_status'].tail(15))

21290          женат / замужем
21291         гражданский брак
21292    не женат / не замужем
21293          женат / замужем
21294    не женат / не замужем
21295          женат / замужем
21296          женат / замужем
21297         гражданский брак
21298          женат / замужем
21299                в разводе
21300         гражданский брак
21301          женат / замужем
21302         гражданский брак
21303          женат / замужем
21304          женат / замужем
Name: family_status, dtype: object

**Вывод**

Дубликаты успешно обработаны, таблица приведена к единообразию. Часть дубликатов возникла, по всей видимости, из-за причин организационного характера, часть из-за невнимательности.

### Лемматизация

Проведем лемматизацию столбца `'purpose'`, чтобы посмотреть, на что чаще всего люди берут кредит:

In [31]:
from pymystem3 import Mystem  # импорт необходимых библиотек
m = Mystem() 

from collections import Counter # импорт необходимых библиотек
print(Counter(m.lemmatize(', '.join(df['purpose']))))



Counter({' ': 33366, ', ': 21304, 'недвижимость': 6311, 'покупка': 5856, 'жилье': 4428, 'автомобиль': 4273, 'образование': 3985, 'с': 2899, 'операция': 2586, 'свадьба': 2308, 'свой': 2216, 'на': 2206, 'строительство': 1869, 'высокий': 1365, 'получение': 1307, 'коммерческий': 1305, 'для': 1285, 'жилой': 1219, 'сделка': 937, 'заниматься': 900, 'дополнительный': 899, 'подержать': 847, 'проведение': 763, 'сыграть': 759, 'сдача': 648, 'семья': 637, 'собственный': 632, 'со': 624, 'ремонт': 604, 'приобретение': 457, 'профильный': 433, 'подержанный': 113, '\n': 1})


Мы видим, что наиболее часто встречающиеся леммы, отражающие суть цели кредита, это покупка жилья, транспорта, получение образования и проведение свадьбы. Выведем итоговую цель кредита в отдельный столбец `'end_purpose'` через функцию и посчитаем количество целей:

In [32]:
df['lemmas_purpose'] = df['purpose'].apply(m.lemmatize)  # столбец с леммами для наглядности
def end_purpose(row):
        if 'недвижимость' in row or 'жилье' in row:
            return 'недвижимость'
        if 'автомобиль' in row:
            return 'автомобиль' 
        if 'свадьба' in row:
            return 'свадьба'
        if 'образование' in row:
            return 'образование'
        return 'другое'    
df['end_purpose'] = df['lemmas_purpose'].apply(end_purpose)
print()
print(df['end_purpose'].value_counts())
print()
display(df.head(10))


недвижимость    10739
автомобиль       4273
образование      3985
свадьба          2308
Name: end_purpose, dtype: int64



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_income,lemmas_purpose,end_purpose
0,1.0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,161404,"[покупка, , жилье, \n]",недвижимость
1,1.0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,161404,"[приобретение, , автомобиль, \n]",автомобиль
2,0.0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,161404,"[покупка, , жилье, \n]",недвижимость
3,3.0,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,161404,"[дополнительный, , образование, \n]",образование
4,0.0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,137120,"[сыграть, , свадьба, \n]",свадьба
5,0.0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,202483,"[покупка, , жилье, \n]",недвижимость
6,0.0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,202483,"[операция, , с, , жилье, \n]",недвижимость
7,0.0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,161404,"[образование, \n]",образование
8,2.0,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,161404,"[на, , проведение, , свадьба, \n]",свадьба
9,0.0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,161404,"[покупка, , жилье, , для, , семья, \n]",недвижимость


**Вывод**

Все многобразие целей кредита мы свели в четыре основных, благодаря лемматизации. Так намного понятнее, зачем люди берут кредит.

### Категоризация данных

Теперь приступим к категоризации данных. Первый столбец для категоризации - `'children'`. Очевидно, что первой категорией будут бездетные люди. Остальных мы поделили на две категории: многодетные и обычные семьи,и сходя из того, что в РФ многодетной семьей считается семья с тремя и более детьми. Посмотрим сначала распределение количества семей в зависимости от количества детей в целом:

In [33]:
print(df['children'].value_counts().head(10))

0.0    14096
1.0     4792
2.0     2039
3.0      328
4.0       41
5.0        9
Name: children, dtype: int64


Напишем функцию для решения задачи категоризации в соответствии с вышеупомянутыми критериями:

In [34]:
def family_type (num):
    if num == 0:
        return 'бездетная'
    if num == 1 or num==2:
        return 'обычная'
    if num>=3:
        return 'многодетная'
    
print(family_type(3))  # проверка работоспособности

многодетная


Добавим столбец в таблицу для наглядности.

In [35]:
df['family_type'] = df['children'].apply(family_type)
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_income,lemmas_purpose,end_purpose,family_type
0,1.0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,161404,"[покупка, , жилье, \n]",недвижимость,обычная
1,1.0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,161404,"[приобретение, , автомобиль, \n]",автомобиль,обычная
2,0.0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,161404,"[покупка, , жилье, \n]",недвижимость,бездетная
3,3.0,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,161404,"[дополнительный, , образование, \n]",образование,многодетная
4,0.0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,137120,"[сыграть, , свадьба, \n]",свадьба,бездетная
5,0.0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,202483,"[покупка, , жилье, \n]",недвижимость,бездетная
6,0.0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,202483,"[операция, , с, , жилье, \n]",недвижимость,бездетная
7,0.0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,161404,"[образование, \n]",образование,бездетная
8,2.0,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,161404,"[на, , проведение, , свадьба, \n]",свадьба,обычная
9,0.0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,161404,"[покупка, , жилье, , для, , семья, \n]",недвижимость,бездетная


Подсчитаем значение в каждой категории:

In [36]:
df['family_type'].value_counts()

бездетная      14096
обычная         6831
многодетная      378
Name: family_type, dtype: int64

Также нам нужно разбить замещиков на категории по уровню дохода. Для начала посмотрим размер максимального, минимального и медианного дохода. Медианный доход выбран из-за очень большой дельты между минимумом и максимумом.

In [37]:
print("Максимальный доход: {}".format(df['total_income'].max()))
print()
print("Минимальный доход: {}".format(df['total_income'].min()))
print()
print("Медианный доход: {}".format(df['total_income'].median()))

Максимальный доход: 2265604

Минимальный доход: 20667

Медианный доход: 151876.0


Согласно статистическим данным, медианный доход в РФ составляет около 40000 рублей. Возьмем все, что ниже, за первую категорию. Следующей планкой мы решили взять 150000, потому что это медианное значение. Далее идут планки 300000 и свыше 300000, так как эти значения позволяют нам не слишком сильно раздувать количество категорий, и при этом получить определенный статистически достоверный материал, потому что экстремумы достаточно немногочисленны по отношению к двум другим категориям. Тем не менее, деление условное и может быть пересмотренно с заданным шагом.

Напишем функцию для категоризации в соответствии с заданными параметрами:

In [38]:
def income_type (num):
    if num < 40000:
        return 'Менее 40000'
    if 40000<num<150000:
        return 'От 40000 до 150000'
    if 150000<num<300000:
        return 'От 150000 до 300000'
    if num>300000:
        return 'Свыше 300000'
    
    
print(income_type (112000))  # тест функции

От 40000 до 150000


Добавим столбец в таблицу для наглядности.

In [39]:
df['income_level'] = df['total_income'].apply(income_type)
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_income,lemmas_purpose,end_purpose,family_type,income_level
0,1.0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,161404,"[покупка, , жилье, \n]",недвижимость,обычная,От 150000 до 300000
1,1.0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,161404,"[приобретение, , автомобиль, \n]",автомобиль,обычная,От 40000 до 150000
2,0.0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,161404,"[покупка, , жилье, \n]",недвижимость,бездетная,От 40000 до 150000
3,3.0,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,161404,"[дополнительный, , образование, \n]",образование,многодетная,От 150000 до 300000
4,0.0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,137120,"[сыграть, , свадьба, \n]",свадьба,бездетная,От 150000 до 300000
5,0.0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,202483,"[покупка, , жилье, \n]",недвижимость,бездетная,От 150000 до 300000
6,0.0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,202483,"[операция, , с, , жилье, \n]",недвижимость,бездетная,От 150000 до 300000
7,0.0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,161404,"[образование, \n]",образование,бездетная,От 40000 до 150000
8,2.0,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,161404,"[на, , проведение, , свадьба, \n]",свадьба,обычная,От 40000 до 150000
9,0.0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,161404,"[покупка, , жилье, , для, , семья, \n]",недвижимость,бездетная,От 40000 до 150000


**Вывод**

Мы провели категоризацю, которой нам недоставало (по уровню дохода, количеству детей в семье) и теперь готовы постараться ответить на вопросы исследования.

## Ответы на вопросы исследования

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Составим сводную таблицу с группировкой по количеству детей в семье, и посмотрим, каков уровень задолженности в каждой из них.

In [40]:
df_children_debts = df.pivot_table(index=['family_type'], values='debt', aggfunc='sum')  # считаем количество должников
df_children_debts['total'] = df.groupby('family_type')['debt'].count()  # считаем, сколько всего людей
df_children_debts['ratio'] = (df_children_debts['total']-df_children_debts['debt'])/df_children_debts['total']  # коэффициент возврата
display(df_children_debts.head(10))


,debt,total,ratio
family_type,,,
бездетная,1066,14096,0.924376
многодетная,31,378,0.917989
обычная,635,6831,0.907041


**Вывод**

Из таблицы мы видим, что люди, не имеющие детей, чаще возвращают кредиты вовремя. Вероятно, это связано с отсутствием лишних затрат на содержание детей и бОльшим количеством свободных средств. Также мы видим, что многодетные семьи лучше возвращают кредиты, чем семьи с одним-двумя детьми. Вероятно, многодетные семьи уже лучше научились распределять финансы и планировать расходы, чем те, для кого дети это новый опыт.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Составим сводную таблицу с группировкой по семейному положению, и посмотрим, каков уровень задолженности в каждой из них.

In [41]:
df_family_debts = df.pivot_table(index=['family_status'], values='debt', aggfunc='sum')  # считаем количество должников
df_family_debts['total'] = df.groupby('family_status')['debt'].count()  # считаем, сколько всего людей
df_family_debts['ratio'] = (df_family_debts['total']-df_family_debts['debt'])/df_family_debts['total']  # коэффициент возврата
display(df_family_debts.head(10))

,debt,total,ratio
family_status,,,
в разводе,85,1181,0.928027
вдовец / вдова,62,950,0.934737
гражданский брак,386,4124,0.906402
женат / замужем,926,12261,0.924476
не женат / не замужем,273,2789,0.902115


**Вывод**

Люди, которые не состоят в браке или состоят в гражданском браке, хуже возвращают кредиты. Семейные люди выплачивают кредиты лучше. 

Также интересно, что разведенные и овдовевшие имеют наилучший коэффициент выплат. Возможно, это происходит потому, что в семейной жизни люди привыкают иметь финансовую подушку и планировать затраты лучше, и когдла остаются одни, продолжают закладывать этот запас на себя по привычке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Составим сводную таблицу с группировкой по категориям дохода, и посмотрим, каков уровень задолженности в каждой из них.

In [42]:
df_income_debts = df.pivot_table(index=['income_level'], values='debt', aggfunc='sum')  # считаем количество должников
df_income_debts['total'] = df.groupby('income_level')['debt'].count()  # считаем, сколько всего людей
df_income_debts['ratio'] = (df_income_debts['total']-df_income_debts['debt'])/df_income_debts['total']  # коэффициент возврата
display(df_income_debts.head(10))

,debt,total,ratio
income_level,,,
Менее 40000,10,123,0.918699
От 150000 до 300000,751,9351,0.919688
От 40000 до 150000,865,10356,0.916474
Свыше 300000,106,1475,0.928136


**Вывод**

Нет ярко выраженной взаимосвязи между уровнем дохода и возвратом кредита в срок. Интересно, что люди с  доходом ниже медианного по России выплачивают кредиты с таким же коэффициентом, как и люди с доходом в несколько раз больше. Вероятно, вопрос самодисциплины. 

Люди с высокими доходами выплачивают кредиты лучше, вероятно, потому что у них просто остаются незадействованные средства.

- Как разные цели кредита влияют на его возврат в срок?

Составим сводную таблицу с группировкой по целям кредита, и посмотрим, каков уровень задолженности в каждой из них.

In [43]:
df_end_purpose_debts = df.pivot_table(index=['end_purpose'], values='debt', aggfunc='sum')  # считаем количество должников
df_end_purpose_debts['total'] = df.groupby('end_purpose')['debt'].count()  # считаем, сколько всего людей
df_end_purpose_debts['ratio'] = (df_end_purpose_debts['total']-df_end_purpose_debts['debt'])/df_end_purpose_debts['total']  # коэффициент возврата
display(df_end_purpose_debts.head(10))


,debt,total,ratio
end_purpose,,,
автомобиль,399,4273,0.906623
недвижимость,779,10739,0.927461
образование,370,3985,0.907152
свадьба,184,2308,0.920277


**Вывод**

Лучше всего выплачивают ипотеку, и это понятно, потому что это очень важная покупка и люди дорожат приобретенным жильем. Удивительно, что люди хорошо выплачивают свадебные кредиты, учитывая статистику разводов в РФ.

Хуже платят автокредиты и образовательные кредиты. Вероятно, в случае с автомобилями сказывается доступность автокредитов, люди берут их, не просчитывая наперед возможные последствия, и потом просто не могут платить. Образовательные кредиты могут платить хуже, потому что люди берут кредиты в расчете найти потом высокооплачиваемую работу, но это удается не всем.

## Шаг 4. Общий вывод

Бездетные люди чаще возвращают кредиты вовремя. Вероятно, это связано с отсутствием лишних затрат на содержание детей и бОльшим количеством свободных средств. 

Многодетные семьи лучше возвращают кредиты, чем семьи с одним-двумя детьми. Вероятно, многодетные семьи уже лучше научились распределять финансы и планировать расходы, чем те, для кого дети это новый опыт.

Люди, которые не состоят в браке или состоят в гражданском браке, хуже возвращают кредиты. Семейные люди выплачивают кредиты лучше. 

Разведенные и овдовевшие имеют наилучший коэффициент выплат. Возможно, это происходит потому, что в семейной жизни люди привыкают иметь финансовую подушку и планировать затраты лучше, и когда остаются одни, продолжают закладывать этот запас на себя по привычке.

Нет ярко выраженной взаимосвязи между уровнем дохода и возвратом кредита в срок. Интересно, что люди с  доходом ниже медианного по России выплачивают кредиты с таким же коэффициентом, как и люди с доходом в несколько раз больше. Вероятно, вопрос самодисциплины. 

Люди с высокими доходами выплачивают кредиты лучше, вероятно, потому что у них просто остаются незадействованные средства.

Лучше всего выплачивают ипотеку, и это понятно, потому что это очень важная покупка и люди дорожат приобретенным жильем. Удивительно, что люди хорошо выплачивают свадебные кредиты, учитывая статистику разводов в РФ.

Хуже платят автокредиты и образовательные кредиты. Вероятно, в случае с автомобилями сказывается доступность автокредитов, люди берут их, не просчитывая наперед возможные последствия, и потом просто не могут платить. Образовательные кредиты могут платить хуже, потому что люди берут кредиты в расчете найти потом высокооплачиваемую работу, но это удается не всем.